#**21 cm Radio Telescope data processing** 
In the video on 21cm radio telescope we saw several plots of 21cm emission from different sources.<br> 
In our discussion we hadn't discussed much about different sources of noise which will be there in the signal due to several factors. Before using the plots for extracting science, they must be processed first.

Following code guides you through how to model noise floor using a polynomial fit and how to use it to remove noise from signal.

In [ ]:
from google.colab import files
uploaded = files.upload()    #"GL180.txt"

In [ ]:
import numpy as np
data=np.loadtxt('GL180.txt')

In [ ]:
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
#Our data is centred at 1420 MHz. Telescope records plus minus 1000 KHz about the central frequency
x=np.linspace(-1000,1000,np.size(data)) 
y=abs(data)


# Original data
print("Original Data")
plt.plot(x,y)
plt.xlabel("Frequency (kHz)")
plt.ylabel("Temperature (K)")
plt.show()

# Removing the signal and keeping the noise floor
y1=y
print("\nAfter removing signal")
for i in range(np.size(data)):
  if(x[i]>-250 and x[i]<250):    #We manually set signal range based on plot. This varies from source to source
    y1[i]=np.NaN
plt.plot(x,y1)
plt.xlabel("Frequency (kHz)")
plt.ylabel("Temperature (K)")
plt.show()

# Replacing null values with median values
print("\nReplacing null values with median values")
y1=y1.reshape(400,1)    # Since imputer function demands multidimensional array

imputer = SimpleImputer(missing_values=np.nan, strategy='median')
y1 = imputer.fit_transform(y1)

y2=np.ravel(y1)            #Back to 1D array


plt.plot(x,y2)
plt.xlabel("Frequency (kHz)")
plt.ylabel("Temperature (K)")
plt.show()

# Modelling the noise floor 
print("\nNoise floor profile (polynomial fit)")

p=np.poly1d(np.polyfit(x,y2,3)) #returns polynomial coefficients a,b,c,d
# p(t)=a*t**3 +bt**2+c*t+d

t=np.linspace(-1000,1000,np.size(data)) 
y_noise=p(t) 

plt.plot(x,y_noise)   
plt.xlabel("Frequency (kHz)")
plt.ylabel("Temperature (K)")
plt.show()

#Removing noise floor from signal
print("\nData after removing noise floor")
y3=data-y_noise
plt.plot(x,y3)
plt.xlabel("Frequency (kHz)")
plt.ylabel("Temperature (K)")
plt.show()
